In [1]:
import sys # for automation and parallelisation
manual, scenario = (True, 'base') if 'ipykernel' in sys.argv[0] else (False, sys.argv[1])
if manual:
    %matplotlib inline


Bad key text.latex.preview in file C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\

In [2]:
import pandas as pd
import numpy as np
from quetzal.model import stepmodel
from quetzal.io import excel

In [3]:
model_path = '../model/' + scenario + '/'
input_path = '../input/'
output_path = '../output/'

In [4]:
# Load parameters for settings
params = excel.read_var(file='../input/parameters.xls', scenario=scenario)

# Trips per person from MiD2017

(Ortuzar 2011 [S. 166], Daly 1997): Define groups of individuals and determine for each group the number of trips per purpose.

Here the group of individuals is given by car availability and the urbanisation degree of the home zone. The deterimination of the number of trip per purpose is done by taking the mean value of number of reported trips per person in MiD2017. 

In [ ]:
# Choose purposes for which volume data will be saved
purposes = ['buy/execute', 'leisure', 'accompany']

In [5]:
# Load data on trips and persons from MiD survey
trips = pd.read_csv(input_path + 'transport_demand/calibration_all_trips_MiD2017.csv',
                    usecols=['P_ID', 'W_HOCH', 'urbanisation', 'purpose_model', 'car_avail',
                             'origin', 'destination'])
persons = pd.read_csv(input_path + 'transport_demand/MiD2017_Regional_Personen.csv', sep=';', decimal=',',
                      usecols=['HP_ID_Reg', 'P_HOCH', 'P_GEW', 'P_RBW_ANZ'])
# Define and rename relevant columns
persons = persons[['HP_ID_Reg', 'P_HOCH', 'P_GEW', 'P_RBW_ANZ']]
persons.columns = ['P_ID', 'P_HOCH', 'weight', 'num_rbw']
# clip person informations to trips table
trips = trips.merge(persons, on='P_ID')

In [6]:
# total trips per day (million)
# Only trips with specified origin and destination and other filters as in cal10
total_trips_MiD = trips['W_HOCH'].sum() / 1e6
total_trips_MiD

67.25827528747712

In [7]:
# Total population of Germany in 2017 (million)
total_pop_MiD = persons['P_HOCH'].sum() / 1e6
total_pop_MiD

82.17568400000002

In [8]:
# drop ureported car availability
trips = trips.loc[trips['car_avail'].isin([0,1])]
# set unreported number of regular business trips (regelmäßige berufliche Wege, rbw) to 0
rbw_dict = {n: 0 for n in [94, 99, 200, 607, 608]}
trips['num_rbw'] = trips['num_rbw'].replace(rbw_dict)
trips.shape

(260709, 10)

In [9]:
# get trips per person for each segment
urban_pop_dict = trips.drop_duplicates('P_ID').groupby('urbanisation').sum()['P_HOCH'].to_dict()
def trips_per_person(g):
    return g['W_HOCH'].sum() / urban_pop_dict[g['urbanisation'].unique()[0]]
segmented_trips_pp = trips.groupby(['urbanisation', 'purpose_model', 'car_avail']
                                  ).apply(trips_per_person)

## Save generation volumes in model zones

In [10]:
sm = stepmodel.read_json(model_path + 'de_zones')

In [ ]:
# Put trip generation in zones table
car_names = {0: '_no_car', 1: '_car'}
for p in purposes:
    for c in [0,1]:
        sm.zones[p+car_names[c]] = [segmented_trips_pp[(u,p,c)] * pop * 365
                                    for u, pop in zip(sm.zones['urbanisation'],
                                                      sm.zones['population'])]

In [ ]:
sm.zones[[p+car_names[c] for p in purposes for c in [0,1]]].to_csv(
    input_path + 'generation_volumes.csv')